### Deliverable 3: Optimize the Model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from keras.callbacks import Callback, ModelCheckpoint
import pandas as pd
import tensorflow as tf
import os

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# Preprocess the Dataset

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine which values to replace 
types = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(types[types < 200].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
classes = application_df['CLASSIFICATION'].value_counts()
# Determine which values to replace
replace_class = list(classes[classes < 1880].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\ajdea\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
merged_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(columns=application_cat)
merged_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Design a Neural Network Model

In [7]:
# Split our preprocessed data into our features and target arrays
y = merged_df['IS_SUCCESSFUL'].values
X = merged_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\ajdea\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


# Optimization Attempt 1: Additional Layers and Neurons

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               5280      
                                                                 
 dense_1 (Dense)             (None, 80)                9680      
                                                                 
 dense_2 (Dense)             (None, 30)                2430      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 17,421
Trainable params: 17,421
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

#Create a callback that saves the model's weight every 5 epochs
callback_5 = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

# Train the model
fit_model = nn.fit(X_train,y_train,callbacks=[callback_5],epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 659us/step - loss: 59120.0938 - accuracy: 0.4864
Epoch 2/100
804/804 [==============================] - 1s 671us/step - loss: 30099.1719 - accuracy: 0.4881
Epoch 3/100
804/804 [==============================] - 1s 661us/step - loss: 9705.5869 - accuracy: 0.4948
Epoch 4/100
804/804 [==============================] - 1s 665us/step - loss: 36932.1836 - accuracy: 0.4947
Epoch 5/100
751/804 [===========================>..] - ETA: 0s - loss: 25757.2539 - accuracy: 0.4984
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 680us/step - loss: 24314.6035 - accuracy: 0.4980
Epoch 6/100
804/804 [==============================] - 1s 659us/step - loss: 8051.0684 - accuracy: 0.5025
Epoch 7/100
804/804 [==============================] - 1s 670us/step - loss: 11417.3682 - accuracy: 0.5062
Epoch 8/100
804/804 [==============================] - 1s 659us/step - loss: 6196.2856 - accuracy: 0.4881
E

804/804 [==============================] - 1s 658us/step - loss: 0.6917 - accuracy: 0.5318
Epoch 64/100
804/804 [==============================] - 1s 664us/step - loss: 0.6915 - accuracy: 0.5318
Epoch 65/100
534/804 [==================>...........] - ETA: 0s - loss: 0.6914 - accuracy: 0.5340
Epoch 00065: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 669us/step - loss: 0.6916 - accuracy: 0.5318
Epoch 66/100
804/804 [==============================] - 1s 664us/step - loss: 0.6915 - accuracy: 0.5318
Epoch 67/100
804/804 [==============================] - 1s 663us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 68/100
804/804 [==============================] - 1s 669us/step - loss: 0.6915 - accuracy: 0.5312
Epoch 69/100
804/804 [==============================] - 1s 663us/step - loss: 0.6916 - accuracy: 0.5307
Epoch 70/100
454/804 [===============>..............] - ETA: 0s - loss: 0.6919 - accuracy: 0.5265
Epoch 00070: saving model to checkpoints\weig

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.0330 - accuracy: 0.3163 - 193ms/epoch - 720us/step
Loss: 1.0329875946044922, Accuracy: 0.3162682354450226


# Optimization Attempt 2: Change Activation of Layers

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 95
hidden_nodes_layer2 = 42

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 95)                4180      
                                                                 
 dense_5 (Dense)             (None, 42)                4032      
                                                                 
 dense_6 (Dense)             (None, 1)                 43        
                                                                 
Total params: 8,255
Trainable params: 8,255
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train,y_train,callbacks=[callback_5],epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 618us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 2/100
804/804 [==============================] - 0s 617us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 3/100
804/804 [==============================] - 1s 624us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 4/100
804/804 [==============================] - 0s 613us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 5/100
328/804 [===========>..................] - ETA: 0s - loss: 8.2077 - accuracy: 0.4679
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 0s 620us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 6/100
804/804 [==============================] - 0s 620us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 7/100
804/804 [==============================] - 0s 612us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 8/100
804/804 [==============================] - 0s 610us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 9/100
804/804 [=============

804/804 [==============================] - 0s 620us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 64/100
804/804 [==============================] - 0s 618us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 65/100
 80/804 [=>............................] - ETA: 0s - loss: 8.2367 - accuracy: 0.4660
Epoch 00065: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 637us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 66/100
804/804 [==============================] - 0s 615us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 67/100
804/804 [==============================] - 0s 607us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 68/100
804/804 [==============================] - 0s 604us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 69/100
804/804 [==============================] - 0s 609us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 70/100
 85/804 [==>...........................] - ETA: 0s - loss: 8.2626 - accuracy: 0.4643
Epoch 00070: saving model to checkpoints\weig

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 8.2422 - accuracy: 0.4657 - 168ms/epoch - 627us/step
Loss: 8.242232322692871, Accuracy: 0.46565598249435425


# Optimization Attempt 3: Remove Noisy Variables

In [14]:
# Drop more columns

In [15]:
merged_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [16]:
trim_df = merged_df.drop(columns=['SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y', 
        'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
        'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional'])
trim_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Split our trimmed data into our features and target arrays
y = trim_df['IS_SUCCESSFUL'].values
X = trim_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\ajdea\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                2480      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4,941
Trainable params: 4,941
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train,y_train,callbacks=[callback_5],epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 600us/step - loss: 89567.7969 - accuracy: 0.4900
Epoch 2/100
804/804 [==============================] - 0s 608us/step - loss: 12352.6514 - accuracy: 0.4782
Epoch 3/100
804/804 [==============================] - 0s 610us/step - loss: 24325.5391 - accuracy: 0.5011
Epoch 4/100
747/804 [==========================>...] - ETA: 0s - loss: 45306.9609 - accuracy: 0.4980
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 0s 615us/step - loss: 45156.7734 - accuracy: 0.4959
Epoch 5/100
804/804 [==============================] - 0s 613us/step - loss: 20595.9492 - accuracy: 0.5012
Epoch 6/100
804/804 [==============================] - 0s 618us/step - loss: 24180.0801 - accuracy: 0.5001
Epoch 7/100
804/804 [==============================] - 0s 610us/step - loss: 16483.8281 - accuracy: 0.4984
Epoch 8/100
804/804 [==============================] - 0s 612us/step - loss: 20961.7422 - accuracy: 0.500

804/804 [==============================] - 0s 595us/step - loss: 0.6913 - accuracy: 0.5318
Epoch 63/100
804/804 [==============================] - 0s 587us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 64/100
506/804 [=================>............] - ETA: 0s - loss: 0.6911 - accuracy: 0.5332
Epoch 00064: saving model to checkpoints\weights.64.hdf5
804/804 [==============================] - 0s 607us/step - loss: 0.6913 - accuracy: 0.5318
Epoch 65/100
804/804 [==============================] - 0s 598us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 66/100
804/804 [==============================] - 0s 593us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 67/100
804/804 [==============================] - 0s 590us/step - loss: 0.6913 - accuracy: 0.5318
Epoch 68/100
804/804 [==============================] - 0s 590us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 69/100
515/804 [==================>...........] - ETA: 0s - loss: 0.6912 - accuracy: 0.5319
Epoch 00069: saving model to checkpoints\weig

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7491 - accuracy: 0.5803 - 166ms/epoch - 620us/step
Loss: 0.7491294145584106, Accuracy: 0.5802915692329407


# Export Results

In [21]:
# Save and export results to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")